In [42]:
import heapq, itertools
from collections import deque

In [43]:
class PriorityQueue():
    
    REMOVED = '<removed-task>'
    
    def __init__(self):
        self.hq = []
        self.entry_finder = {}
        self.counter = itertools.count()
    
    def add_with_priority(self, task, priority):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove_task(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heapq.heappush(self.hq, entry)
    
    def remove_task(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED
    
    def extract_min(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.hq:
            priority, count, task = heapq.heappop(self.hq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task, priority
        raise KeyError('pop from an empty priority queue')

In [44]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

In [45]:
random.seed(30)

In [46]:
G = nx.random_geometric_graph(20, 0.42)
pos = nx.get_node_attributes(G, 'pos')

In [47]:
def Dijkstra(G, s):
    """
    A more efficient implementation for Dijkstra's algorithm using
    min-priority queue.
    
    Parameters
    ----------
    G : NetworkX Graph
    
    s : source node label
    
    Returns
    -------
    path: dict
        Paths from the source.
    dist: dict
        Distances from source node to any other nodes in the graph.
    """
    # Keys are node labels, values are distances from the source to the corresponding
    # nodes, which will be updated during the algorithm.
    dist = {}
    # Keys are node labels, values are previous node in the shortest path from source
    prev = {}
    
    Q = PriorityQueue() # Unvisited nodes
    # Step 1
    Q.add_with_priority(s, 0)
    for v in G:
        dist[v] = float('inf')
    dist[s] = 0
    
    procs = []
    while len(Q.hq):
        try:
            # Step 2, step 5
            # node with the smallest assigned distance will be choosed
            # thanks to min-priority queue, the operation runs in O(1) in this implementation
            # since extract_min remove the lowest priority task from the queue,
            # this operation also implements step 4
            u, dist_u = Q.extract_min()
            tmp = prev.get(u)
            if not tmp is None:
                procs.append((u, tmp, 'r'))
        except KeyError:
            # Step 5, stop condition
            break
        
        # Step 3
        for v in G[u]:
            alt = dist[u] + G[u][v]['weight']
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u
                Q.add_with_priority(v, alt)
                procs.append((u, v, 'b'))
    
    return procs

In [48]:
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

edges = [e+(euclidean_distance(pos[e[0]],pos[e[1]]),) for e in G.edges_iter()]
G.add_weighted_edges_from(edges)
procs = Dijkstra(G, 13)

In [49]:
procs

[(13, 0, 'b'),
 (13, 3, 'b'),
 (13, 8, 'b'),
 (13, 9, 'b'),
 (13, 12, 'b'),
 (13, 14, 'b'),
 (13, 16, 'b'),
 (13, 19, 'b'),
 (12, 13, 'r'),
 (12, 5, 'b'),
 (3, 13, 'r'),
 (3, 1, 'b'),
 (3, 7, 'b'),
 (3, 11, 'b'),
 (3, 15, 'b'),
 (14, 13, 'r'),
 (8, 13, 'r'),
 (19, 13, 'r'),
 (19, 7, 'b'),
 (19, 11, 'b'),
 (16, 13, 'r'),
 (0, 13, 'r'),
 (0, 2, 'b'),
 (0, 6, 'b'),
 (0, 10, 'b'),
 (0, 17, 'b'),
 (9, 13, 'r'),
 (9, 4, 'b'),
 (7, 19, 'r'),
 (15, 3, 'r'),
 (15, 18, 'b'),
 (5, 12, 'r'),
 (1, 3, 'r'),
 (11, 19, 'r'),
 (4, 9, 'r'),
 (10, 0, 'r'),
 (2, 0, 'r'),
 (17, 0, 'r'),
 (18, 15, 'r'),
 (6, 0, 'r')]

In [50]:
fig = plt.figure()
ax = fig.add_subplot(111)


In [51]:
def init():
    nx.draw(G, pos, node_color='w', node_size=100, alpha=0.5)
    return ax

def update(i):
    covers = [sorted(tmp[:2]) for tmp in procs[:i] if tmp[2]=='r']
    edge_colors = []
    widths = []
    for edge in G.edges():
        if sorted(edge) in covers:
            edge_colors.append('r')
            widths.append(2)
        else:
            if sorted(edge)==sorted(procs[i][:2]):
                edge_colors.append(procs[i][2])
                widths.append(2)
            else:
                edge_colors.append('k')
                widths.append(1)
    ax.clear()
    nx.draw(G, pos, node_color='w', node_size=100, edge_color=edge_colors, width=widths)
    return ax

animation = FuncAnimation(fig, update, range(len(procs)), interval=800, init_func=init)

In [52]:
HTML(animation.to_html5_video())